In [16]:
import re

# Fields we want to capture
KEEP_FIELDS = {
    "CCSDS_CDM_VERS", "CREATION_DATE", "ORIGINATOR",
    "OBJECT_NAME", "INTERNATIONAL_DESIGNATOR", "MANEUVERABLE",
    "REF_FRAME", "X", "Y", "Z"
}

def clean_value(val):
    """Remove units like [km], [m], [m/s] and strip whitespace."""
    if val is None:
        return None
    val = re.sub(r'\[.*?\]', '', val)
    val = val.strip()
    return val if val else None

def parse_first_object_kvn(file_path):
    data = {
        "CIC_OEM_VERS": None,
        "CREATION_DATE": None,
        "ORIGINATOR": None,
        "OBJECT_NAME": None,
        "OBJECT_ID": None,
        "REF_FRAME": "EME2000",
        "MANEUVERABLE": "YES",
        "Epoch": None,
        "X": None,
        "Y": None,
        "Z": None
    }

    xyz_captured = {"X": False, "Y": False, "Z": False}

    with open(file_path, "r") as f:
        for line in f:
            line = line.strip()
            if not line or line.startswith("COMMENT"):
                continue

            if "=" not in line:
                continue

            key, val = line.split("=", 1)
            key = key.strip()
            val = clean_value(val)

            # Capture first X/Y/Z
            if key in xyz_captured:
                if not xyz_captured[key] and val is not None:
                    data[key] = val
                    xyz_captured[key] = True
                continue

            # Top-level fields
            if key == "CCSDS_CDM_VERS":
                data["CIC_OEM_VERS"] = val
            elif key == "CREATION_DATE":
                data["CREATION_DATE"] = val
                if not data["Epoch"]:
                    data["Epoch"] = val
            elif key == "ORIGINATOR":
                data["ORIGINATOR"] = val
            elif key == "OBJECT_NAME":
                data["OBJECT_NAME"] = val
            elif key == "INTERNATIONAL_DESIGNATOR":
                data["OBJECT_ID"] = val
            elif key == "MANEUVERABLE":
                if val is None or val.strip() == "" or val.lower() == "nan" or val.upper() != "NO":
                    data["MANEUVERABLE"] = "YES"
                else:
                    data["MANEUVERABLE"] = val
            elif key == "REF_FRAME":
                data["REF_FRAME"] = "EME2000"  # force

    return data

def write_oem_file(data, output_path):
    with open(output_path, "w") as f:
        f.write(f"CIC_OEM_VERS = {data['CIC_OEM_VERS']}\n")
        f.write(f"CREATION_DATE = {data['CREATION_DATE']}\n")
        f.write(f"ORIGINATOR = {data['ORIGINATOR']}\n")
        f.write("META_START\n")
        f.write(f"OBJECT_NAME = {data['OBJECT_NAME']}\n")
        f.write(f"OBJECT_ID = {data['OBJECT_ID']}\n")
        f.write("CENTER_NAME = EARTH\n")
        f.write(f"REF_FRAME = {data['REF_FRAME']}\n")
        f.write("TIME_SYSTEM = UTC\n")
        f.write("META_STOP\n")
        f.write(f"Epoch = {data['Epoch']}\n")
        f.write(f"X = {data['X']}\n")
        f.write(f"Y = {data['Y']}\n")
        f.write(f"Z = {data['Z']}\n")

    print(f"Saved OEM file to {output_path}")
    print("First X, Y, Z captured:", data["X"], data["Y"], data["Z"])

# ----------------------
# MAIN
# ----------------------
input_file = "/content/CDM_5_10520_sms9_11282159.kvn"
output_file = "OEM_output_with_meta.oem"

data = parse_first_object_kvn(input_file)
write_oem_file(data, output_file)


Saved OEM file to OEM_output_with_meta.oem
First X, Y, Z captured: 5800.196907132434 2627.371355189501 3265.84622687388
